In [59]:
#---------------- import ------------------
from ngsolve import *
from netgen.csg import *
import netgen.gui
from netgen.meshing import ImportMesh
from netgen.meshing import FaceDescriptor
import numpy as np
from scipy.integrate import quad

In [2]:
#---------------- meshing ------------------
geo = CSGeometry()
sph = Sphere(Pnt(0, 0, 0.2), 1).bc("sph")
down = Plane(Pnt(0, 0, 0), Vec(0, 0, -1)).bc("down")
top = Plane(Pnt(0, 0, 1.1), Vec(0, 0, -1)).bc("top")
sphtop = sph * top
sphdown = sph * down
pt = sphdown - sphtop 
geo.AddSurface(sph, pt)
geo.AddSurface(down, pt)
geo.AddSurface(top, pt)
ngmesh = geo.GenerateMesh(maxh=0.1)
mesh = Mesh(ngmesh)
mesh.Curve(2)
Draw(mesh)

 Start Findpoints
 Analyze spec points
 Find edges
 Start Findpoints
 Analyze spec points
 Find edges
 Start Findpoints
 Analyze spec points
 Find edges
 Surface 1 / 3
 Optimize Surface
 Surface 2 / 3
 Optimize Surface
 Surface 3 / 3
 Optimize Surface
 Remove Illegal Elements
 Volume Optimization
 Curve elements, order = 2


In [3]:
#---------------- parameters ------------------
Pi = np.pi
orderset = 2
dimset = mesh.dim
v = 0.5
Yg = 1
Yf = 1
Pout = 1
def rho(theta):
  return np.sin(10*pi*theta)
def integrand0(theta):
    return rho(theta)
rho0value, _ = quad(lambda theta: integrand0(theta).real, 0, np.pi)
rho0value = rho0value*2
def integrand1(theta):
    return rho(theta) * np.exp(-2j * theta)
rho1value, _ = quad(lambda theta: integrand1(theta).real, 0, np.pi)
rho1bvalue, _ = quad(lambda theta: integrand1(theta).imag, 0, np.pi)
rho1value = rho1value*2
rho1bvalue = - rho1bvalue*2
def integrand2(theta):
    return rho(theta) * np.exp(-4j * theta)
rho2value, _ = quad(lambda theta: integrand2(theta).real, 0, np.pi)
rho2bvalue, _ = quad(lambda theta: integrand2(theta).imag, 0, np.pi)
rho2value = rho2value*2
rho2bvalue = - rho2bvalue*2

In [4]:
#---------------- FESpace ------------------    
fes = H1(mesh, order=orderset,  dirichlet="down|top", dim=dimset)
u = fes.TrialFunction()
nsurf = specialcf.normal(3)

In [5]:
#-------------------- energy function --------------------------
u_grad = grad(u).Trace()
Sgradu = u_grad - u_grad * OuterProduct(nsurf, nsurf)
Eu = 1/2*(Sgradu.trans * Sgradu - Id(dimset))
# def Cmatrix(rho0, rho1,rho1b,rho2,rho2b):
#   Cmat = Yg*(1*(globals()['E00']+globals()['E11']) + v*(globals()['E10']+globals()['E01']) + (1-v)/2*globals()['E22']) + Pi*Yf/16*((3*rho0+rho2+4*rho1)*globals()['E00']\
#   +(3*rho0+rho2-4*rho1)*globals()['E11']+(rho0-rho2)*(globals()['E01']+globals()['E10']+globals()['E22'])+(2*rho1b+rho2b)*(globals()['E02']+globals()['E20'])\
#   +(2*rho1b-rho2b)*(globals()['E12']+globals()['E21']))
#   return Cmat       
Cg = CoefficientFunction(((Yg,v*Yg,0),(Yg,v*Yg,0),(0,0,Yg*(v-1)/2)),dims=(3,3))
# Phi = Eu.trans * (Cg * Eu)
Phi = Eu.trans * (Eu)
beta = Pout*nsurf
ngradu = u_grad.trans * nsurf
a = BilinearForm(fes, symmetric=False, check_unused=False)
a += Variation(Trace(Phi).Compile()*ds)
a += Variation((ngradu-beta)*(ngradu-beta)*ds)

In [56]:
#------------------- grid functions ---------------------
u = GridFunction(fes)
du = GridFunction(fes)
res = u.vec.CreateVector()
ini = CoefficientFunction([sin(x**2*y**2)*z*(1.1-z)*nsurf if bc == 'sph' else (0,0,0) for bc in mesh.GetBoundaries()])
u.Set(ini,definedon=fes.mesh.Boundaries("sph|down|top"))
Draw(u)

In [58]:
#------------------- Newton ---------------------
step = 0.2
for i in range(5):
   print ("Newton iteration", i+1)
   print ("  energy = ", a.Energy(u.vec))
   a.Apply(u.vec, res) 
   a.AssembleLinearization(u.vec)
   du.vec.data = a.mat.Inverse(fes.FreeDofs()) * res
   L2norm = sqrt(Integrate(InnerProduct(du,du), mesh, definedon=fes.mesh.Boundaries("sph")))
#    du.vec.data *= 1/L2norm
   print ("  err^2 =", InnerProduct(du.vec, res))
   u.vec.data += step * du.vec.data
Draw(u)

Newton iteration 1
  energy =  18.389517469167206
  err^2 = -0.05731799975444444
Newton iteration 2
  energy =  18.376915024056196
  err^2 = -0.0840061034780522
Newton iteration 3
  energy =  18.358450612630815
  err^2 = -0.1243325492771966
Newton iteration 4
  energy =  18.331137461922474
  err^2 = -0.1872321769034334
Newton iteration 5
  energy =  18.29004953025849
  err^2 = -0.29214523075438237
